In [1]:
from random import seed, randint
from numpy import array, argmax
from math import ceil, log10, sqrt
from keras.models import Sequential
from keras.layers import Dense, LSTM, TimeDistributed, RepeatVector

In [2]:
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y

In [3]:
x, y = random_sum_pairs(100, 3, 15)
print(len(x))
print(x[0])
print(y[0])

100
[11, 11, 8]
30


In [4]:
def to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest + 1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = "+".join([str(n) for n in pattern])
        strp = "".join([" " for _ in range(max_length - len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest + 1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = "".join([" " for _ in range(max_length - len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr

In [5]:
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
    yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc

In [6]:
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc

In [7]:
def generate_data(n_samples, n_numbers, largest, alphabet):
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    X, y = to_string(X, y, n_numbers, largest)
    X, y = integer_encode(X, y, alphabet)
    X, y = one_hot_encode(X, y, len(alphabet))
    X, y = array(X), array(y)
    return X, y

In [8]:
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return "".join(strings)

In [9]:
seed(1)
n_samples = 1000
n_numbers = 2
largest = 10
alphabet = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "+", "-", "*", "/", " "]
n_chars = len(alphabet)
n_in_seq_length = n_numbers * ceil(log10(largest + 1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest + 1)))

In [10]:
X, y = generate_data(n_samples, n_numbers, largest, alphabet)

In [11]:
print("shape of X", X.shape)
print("shape of y", y.shape)
print("X[0]:")
print(X[0])
print("y[0]")
print(y[0])

print("invert X[0]", invert(X[0], alphabet) )
print("invert y[0]", invert(y[0], alphabet) )

shape of X (1000, 5, 15)
shape of y (1000, 2, 15)
X[0]:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
y[0]
[[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]]
invert X[0]  3+10
invert y[0] 13


In [12]:
n_batch = 10
n_epoch = 30

In [13]:
model = Sequential()
model.add(LSTM(100, input_shape = (n_in_seq_length, n_chars)))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences = True))
model.add(TimeDistributed(Dense(n_chars, activation= "softmax")))
model.compile(loss= "categorical_crossentropy", optimizer= "adam", metrics= ["accuracy"])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               46400     
                                                                 
 repeat_vector (RepeatVecto  (None, 2, 100)            0         
 r)                                                              
                                                                 
 lstm_1 (LSTM)               (None, 2, 50)             30200     
                                                                 
 time_distributed (TimeDist  (None, 2, 15)             765       
 ributed)                                                        
                                                                 
Total params: 77365 (302.21 KB)
Trainable params: 77365 (302.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [14]:
for i in range(n_epoch):
    X, y = generate_data(n_samples, n_numbers, largest, alphabet)
    print(i)
    model.fit(X, y, epochs = 1, batch_size = n_batch)

0
100/100 [==============================] - 7s 9ms/step - loss: 2.1104 - accuracy: 0.3445
1
100/100 [==============================] - 1s 8ms/step - loss: 1.5421 - accuracy: 0.3655
2
100/100 [==============================] - 1s 9ms/step - loss: 1.3869 - accuracy: 0.4590
3
100/100 [==============================] - 1s 9ms/step - loss: 1.2872 - accuracy: 0.5160
4
100/100 [==============================] - 1s 9ms/step - loss: 1.1871 - accuracy: 0.5625
5
100/100 [==============================] - 1s 9ms/step - loss: 1.0716 - accuracy: 0.6190
6
100/100 [==============================] - 1s 9ms/step - loss: 0.9105 - accuracy: 0.6780
7
100/100 [==============================] - 1s 9ms/step - loss: 0.7541 - accuracy: 0.7685
8
100/100 [==============================] - 1s 9ms/step - loss: 0.6432 - accuracy: 0.8520
9
100/100 [==============================] - 1s 9ms/step - loss: 0.5424 - accuracy: 0.9100
10
100/100 [==============================] - 1s 9ms/step - loss: 0.4692 - accuracy: 0.928

In [15]:
X, y = generate_data(n_samples, n_numbers, largest, alphabet)
result = model.predict(X, batch_size = n_batch, verbose = 0)
expected = [invert(x, alphabet) for x in y]
predicted = [invert(x, alphabet) for x in result]
for i in range(20):
    print("Expected=%s, Predicted=%s" % (expected[i], predicted[i]))

Expected=15, Predicted=15
Expected=12, Predicted=12
Expected=12, Predicted=12
Expected=13, Predicted=13
Expected=15, Predicted=15
Expected= 8, Predicted= 8
Expected= 4, Predicted= 4
Expected=19, Predicted=19
Expected=13, Predicted=13
Expected=15, Predicted=15
Expected= 8, Predicted= 8
Expected=13, Predicted=13
Expected= 4, Predicted= 4
Expected=16, Predicted=16
Expected=13, Predicted=13
Expected=16, Predicted=16
Expected= 8, Predicted= 8
Expected=14, Predicted=14
Expected=16, Predicted=16
Expected= 3, Predicted= 3
